In [ ]:
# import sys
# import gymnasium as gym

# sys.modules["gym"] = gym
import gym
gym.__version__

In [ ]:
import dataclasses

from stable_baselines3 import PPO
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.vec_env import DummyVecEnv, SubprocVecEnv, VecVideoRecorder, VecFrameStack
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.callbacks import EvalCallback
from stable_baselines3.common.policies import ActorCriticPolicy

from imitation.data import rollout
from imitation.data.wrappers import RolloutInfoWrapper
from imitation.algorithms import bc

from MountainCar_utils.tcn_extractor import TCNExtractor
from MountainCar_utils.observation_wrapper import MountainCarContinuousObsWrapper
from MountainCar_utils.observation_wrapper import MountainCarContinuousNoVelObsWrapper

In [ ]:
import numpy as np
import torch as th
from torch import nn
import matplotlib.pyplot as plt
from pprint import pprint

In [ ]:
import gnwrapper

In [ ]:
# expert = PPO.load("./policy/ppo_MountainCarContinuous_expert3464.zip", print_system_info=True)
expert = PPO.load("./policy/ppo_MountainCarContinuous_expert46304_n32b64.zip", print_system_info=True)

In [ ]:
env_id = "MountainCarContinuous-v0"
NUM_CPU = 32  # Number of processes to use

# Simple BC

In [ ]:
def make_env(env_id):
    def _init():
        env = gym.make(env_id)
        env = RolloutInfoWrapper(env) # Wrapper to save origin obs
        env = MountainCarContinuousObsWrapper(env) # Wrapper Obs
        return env
    
    return _init

bc_env = DummyVecEnv([make_env(env_id)]*NUM_CPU)
bc_env.observation_space

In [ ]:
NUM_EPISODES = 1000
rng = np.random.default_rng()
rollouts = rollout.rollout(
    expert,
    bc_env,
    rollout.make_sample_until(min_timesteps=None, min_episodes=NUM_EPISODES),
    rng = rng,
    unwrap = True,
)

bc_env.close()

In [ ]:
top_rollouts = [rollout for rollout in rollouts if sum(rollout.rews) > 94]
transitions = rollout.flatten_trajectories(top_rollouts)

In [ ]:
pprint(rollout.rollout_stats(rollouts))
pprint(rollout.rollout_stats(top_rollouts))

In [ ]:
print(
    f"""The `rollout` function generated a list of {len(top_rollouts)} {type(top_rollouts[0])}.
After flattening, this list is turned into a {type(transitions)} object containing {len(transitions)} transitions.
The transitions object contains arrays for: {', '.join(transitions.__dict__.keys())}."
"""
)

In [ ]:
env = gym.make(env_id)
env = DummyVecEnv([lambda: env])

bc_trainer = bc.BC(
    observation_space=env.observation_space,
    action_space=env.action_space,
    demonstrations=transitions,
    rng=rng,
    policy=ActorCriticPolicy(observation_space=env.observation_space,
                             action_space=env.action_space,
                             lr_schedule=lambda _: th.finfo(th.float32).max,
                             net_arch=[64, 64]
                             )
)

In [ ]:
bc_trainer.train(n_epochs=10)

# TCN BC

In [ ]:
def make_tcn_env(env_id):
    def _init():
        env = gym.make(env_id)
        env = MountainCarContinuousNoVelObsWrapper(env)
        env = gym.wrappers.FrameStack(env, 4)
        env = RolloutInfoWrapper(env) # Wrapper to save origin obs
        env = MountainCarContinuousObsWrapper(env) # Wrapper Obs
        return env
    
    return _init

bc_tcn_env = DummyVecEnv([make_tcn_env(env_id)]*NUM_CPU)
bc_tcn_env.observation_space

In [ ]:
NUM_EPISODES = 1000
rng = np.random.default_rng()
rollouts = rollout.rollout(
    expert,
    bc_tcn_env,
    rollout.make_sample_until(min_timesteps=None, min_episodes=NUM_EPISODES),
    rng = rng,
    unwrap = True,
)

bc_tcn_env.close()

In [ ]:
top_rollouts = [rollout for rollout in rollouts if sum(rollout.rews) > 94]
transitions_novel = rollout.flatten_trajectories(top_rollouts)

In [ ]:
pprint(rollout.rollout_stats(rollouts))
pprint(rollout.rollout_stats(top_rollouts))

In [ ]:
print(
    f"""The `rollout` function generated a list of {len(top_rollouts)} {type(top_rollouts[0])}.
After flattening, this list is turned into a {type(transitions_novel)} object containing {len(transitions_novel)} transitions.
The transitions object contains arrays for: {', '.join(transitions_novel.__dict__.keys())}."
"""
)

In [ ]:
env_novel = gym.make(env_id)
env_novel = MountainCarContinuousNoVelObsWrapper(env_novel)
env_novel = DummyVecEnv([lambda: env_novel])
env_novel = VecFrameStack(env_novel, 4)

bc_trainer_novel = bc.BC(
    observation_space=env_novel.observation_space,
    action_space=env_novel.action_space,
    demonstrations=transitions_novel,
    rng=rng,
    batch_size=32,
    policy=ActorCriticPolicy(observation_space=env_novel.observation_space,
                             action_space=env_novel.action_space,
                             lr_schedule=lambda _: th.finfo(th.float32).max,
                             net_arch=[64, 64],
                             ortho_init=False,
                             features_extractor_class=TCNExtractor,
                             features_extractor_kwargs=dict(input_size = 1),
                             # sde=True,
                             )
)

In [ ]:
bc_trainer_novel.train(n_epochs=10)

# test BC policy

In [ ]:
from stable_baselines3.common.evaluation import evaluate_policy

# test_env_angle = make_vec_env(env_id, wrapper_class=wrapper, n_envs=10)
# test_env = make_vec_env(env_id, n_envs=10)
# test_env_novel = make_vec_env(env_id, wrapper_class=MountainCarContinuousNoVelObsWrapper, n_envs=10)
# test_env_novel = VecFrameStack(test_env_novel, 4)

test_env_novel = gym.make(env_id)
test_env_novel = MountainCarContinuousNoVelObsWrapper(test_env_novel)
test_env_novel = DummyVecEnv([lambda: test_env_novel])
test_env_novel = VecFrameStack(test_env_novel, 4)

# test_env_angle.seed(0)
# expert_reward, expert_reward_std = evaluate_policy(expert, test_env_angle, 1000)
# test_env.seed(0)
# bc_reward, bc_reward_std = evaluate_policy(bc_trainer.policy, test_env, 1000)
test_env_novel.seed(0)
bc_novel_reward, bc_novel_reward_std = evaluate_policy(bc_trainer_novel.policy, test_env_novel, 100)
# noob_reward, noob_reward_std = evaluate_policy(noob, test_env, 10)

In [ ]:
# print(f'expert reward {expert_reward:.2f} +/- {expert_reward_std:.2f}')
# print(f'BC reward {bc_reward:.2f} +/- {bc_reward_std:.2f}')
print(f'BC noVel reward {bc_novel_reward:.2f} +/- {bc_novel_reward_std:.2f}')
# print(f'noob reward {noob_reward} +/- {noob_reward_std}')

# Save results

In [ ]:
bc_trainer.save_policy('policy/bc_expert3464.zip')
# bc_trainer.save_policy('policy/bc_expert46304_n32b64.zip')

In [ ]:
bc_trainer_novel.save_policy('policy/bc_tcn_expert46304_n32b64.zip')

# Animate

In [ ]:
env = gym.make(env_id)
env = gnwrapper.Animation(env)
# env = gym.wrappers.RecordVideo(env, 'video')
# env = make_vec_env(env_id, wrapper_class=wrapper_image, n_envs=1)
# env = VecVideoRecorder(env, './video', True, 1000)

obs = env.reset()
while True:
    action, _states = bc_trainer.policy.predict(np.asarray(obs))
    obs, rewards, dones, info = env.step(action)
    env.render()
    if dones:
        break
    
env.close()

In [ ]:
env = gym.make(env_id)
env = wrapper(env)
env = gnwrapper.Animation(env)
# env = wrapper(env)
# env = gym.wrappers.RecordVideo(env, 'video')
# env = make_vec_env(env_id, wrapper_class=wrapper, n_envs=1)
# env = VecVideoRecorder(env, './video', lambda x: True, 1000)

obs = env.reset()
while True:
    action, _states = expert.predict(obs.copy())
    obs, rewards, dones, info = env.step(action)
    env.render()
    if dones:
        break
    
env.close()